In [3]:
# Imports (External)
import numpy as np
import pandas as pd
import datetime
from datetime import datetime

from monthdelta import monthdelta
import xlrd
import xlsxwriter
from collections import OrderedDict
import pickle

import sys
sys.path.append('../')  

# Visualization/plotting imports
import matplotlib as mpl
import matplotlib.pyplot as plt

# Machine learning imports
import pywt
from pywt import wavedec, waverec
from scipy import signal
from statsmodels.robust import mad

import sklearn
import tensorflow as tf
import keras
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import optimizers
from keras import regularizers

# Internal Imports
from wsae_lstm.utils import pickle_load

In [4]:
dict_dataframes_index=pickle_load(path_filename="../data/interim/clean_data_index_interval.pickle")
#dict_dataframes_index.keys()

In [5]:
#dict_dataframes_index['csi300 index data'][1].drop(['date'],axis=1,inplace=True)

In [6]:
#dict_dataframes_index['csi300 index data'][1].reset_index(drop=True,inplace=True)

In [7]:
dict_dataframes_index['csi300 index data'][1].head()

,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,
2008-07-01,2799.20,2809.38,2690.18,2698.35,288515.85,72.34,2.5006,-213.078565,-128.949052,119.200,3173.475692,3033.748201,2851.2504,-91.594,2851.3262,-280.770,-9.424605,-0.097927,-8.726278e+07
2008-07-02,2702.63,2745.94,2670.06,2699.60,279163.65,71.99,2.7238,-213.732249,-139.719688,75.875,3140.413385,3001.924277,2822.0828,-152.318,2797.3382,-252.645,-8.557733,-0.026497,-1.095493e+08
2008-07-03,2654.48,2807.68,2617.26,2760.61,456603.05,72.73,2.5762,-206.941406,-112.113057,190.424,3109.044731,2978.942155,2820.8364,-208.925,2753.2792,-82.064,-2.886857,-0.031251,-5.855737e+07
2008-07-04,2751.21,2783.85,2716.02,2741.85,379050.10,72.71,2.5632,-200.759162,-81.997539,67.825,3073.107115,2956.362140,2810.0548,-239.055,2738.4454,-249.421,-8.338289,-0.017237,-7.467503e+07
2008-07-07,2747.61,2890.99,2747.61,2882.76,527320.24,72.71,2.5679,-182.386907,21.707767,149.133,3046.256923,2949.352699,2819.3370,66.742,2756.6342,109.687,3.955427,0.010701,-3.531151e+07


In [8]:
dict_dataframes_index['csi300 index data'][1].head()

,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,
2008-07-01,2799.20,2809.38,2690.18,2698.35,288515.85,72.34,2.5006,-213.078565,-128.949052,119.200,3173.475692,3033.748201,2851.2504,-91.594,2851.3262,-280.770,-9.424605,-0.097927,-8.726278e+07
2008-07-02,2702.63,2745.94,2670.06,2699.60,279163.65,71.99,2.7238,-213.732249,-139.719688,75.875,3140.413385,3001.924277,2822.0828,-152.318,2797.3382,-252.645,-8.557733,-0.026497,-1.095493e+08
2008-07-03,2654.48,2807.68,2617.26,2760.61,456603.05,72.73,2.5762,-206.941406,-112.113057,190.424,3109.044731,2978.942155,2820.8364,-208.925,2753.2792,-82.064,-2.886857,-0.031251,-5.855737e+07
2008-07-04,2751.21,2783.85,2716.02,2741.85,379050.10,72.71,2.5632,-200.759162,-81.997539,67.825,3073.107115,2956.362140,2810.0548,-239.055,2738.4454,-249.421,-8.338289,-0.017237,-7.467503e+07
2008-07-07,2747.61,2890.99,2747.61,2882.76,527320.24,72.71,2.5679,-182.386907,21.707767,149.133,3046.256923,2949.352699,2819.3370,66.742,2756.6342,109.687,3.955427,0.010701,-3.531151e+07


In [9]:
len(dict_dataframes_index['csi300 index data'][1])

612

In [10]:
len(dict_dataframes_index['csi300 index data'][1].columns)

19

In [11]:
len(dict_dataframes_index['csi300 index data'])

24

In [12]:
# train
dict_dataframes_index['csi300 index data'][1].index[0]

Timestamp('2008-07-01 00:00:00')

In [13]:
# validate
dict_dataframes_index['csi300 index data'][1].index[0] + monthdelta(24) 

Timestamp('2010-07-01 00:00:00')

In [14]:
# test
dict_dataframes_index['csi300 index data'][1].index[0] + monthdelta(27) 

Timestamp('2010-10-01 00:00:00')

In [ ]:
def interval_split(df):
    """Split dataframe contents into train/validate/test intervals as defined
    in Bao, Yue, Rao (2017): 24 interval split."""
    dict_dataframes = {}
    split_count = 24
    month_increment = 0
    interval_index = 1
    
    df = df.set_index(df['date'])
    while split_count > 0:
        front = df['date'][0] 
        front = front + monthdelta(month_increment)
        back = df['date'][0]+monthdelta(30) 
        back = back + monthdelta(month_increment)
        month_increment += 3
        split_count -= 1
        df_interval = pd.DataFrame(df[(df['date'] >= front) & (df['date'] <= back)])
        dict_dataframes[interval_index] = df_interval
        dict_dataframes[interval_index].drop(['date'],axis=1,inplace=True)

        interval_index += 1
        #print(front,back)
    return dict_dataframes

In [46]:
def tvt_split(df):
    train = df.index[0]
    validate = df.index[0] + monthdelta(24) 
    test = df.index[0] + monthdelta(27) 
    test_end = df.index[0] + monthdelta(30) 
    

    print("Train start:\n",train)
    print("Train end/validate start:\n",validate)
    print("Validate end/test start:\n",test)
    print("Test end:\n",test_end)
    
    df_train = pd.DataFrame(df[(df.index >= train) & (df.index <= validate)])
    df_validate = pd.DataFrame(df[(df.index >= validate) & (df.index <= test)])
    df_test = pd.DataFrame(df[(df.index >= test) & (df.index <= test_end)])
    return df_train,df_validate,df_test

In [47]:
train,validate,test = tvt_split(dict_dataframes_index['csi300 index data'][1])

Train start:
 2008-07-01 00:00:00
Train end/validate start:
 2010-07-01 00:00:00
Validate end/test start:
 2010-10-01 00:00:00
Test end:
 2011-01-01 00:00:00


In [49]:
train.head(2)

,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,
2008-07-01,2799.20,2809.38,2690.18,2698.35,288515.85,72.34,2.5006,-213.078565,-128.949052,119.200,3173.475692,3033.748201,2851.2504,-91.594,2851.3262,-280.770,-9.424605,-0.097927,-8.726278e+07
2008-07-02,2702.63,2745.94,2670.06,2699.60,279163.65,71.99,2.7238,-213.732249,-139.719688,75.875,3140.413385,3001.924277,2822.0828,-152.318,2797.3382,-252.645,-8.557733,-0.026497,-1.095493e+08


In [50]:
validate.head(2)

,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,
2010-07-01,2557.92,2574.10,2524.57,2526.07,328576.46,84.55,2.2492,-68.405561,-199.061153,49.525,2742.562654,2713.585325,2691.0766,-232.424,2626.8448,-223.952,-8.143641,-0.063088,-9.298846e+07
2010-07-02,2523.04,2542.12,2462.20,2534.11,449141.00,84.37,2.1942,-74.447502,-160.377588,79.920,2729.510192,2696.491961,2674.8705,-223.393,2586.4084,-224.761,-8.146862,-0.088669,-8.628590e+07


In [51]:
test.head(2)

,open,high,low,close,volume,us dollar index,shibor,macd,cci,atr,boll,ema20,ma10,mtm6,ma5,mtm12,roc,smi,wvad
date,,,,,,,,,,,,,,,,,,,
2010-10-08,2965.65,3055.84,2963.24,3044.23,999947.20,77.20,1.7325,14.071181,187.594016,120.262,2912.040962,2913.539251,2898.0211,194.405,2928.1110,111.679,3.808250,0.043045,1.079573e+08
2010-10-11,3069.19,3154.20,3069.19,3132.90,1587349.04,77.47,1.6683,29.910782,243.000652,109.971,2920.543577,2934.430275,2919.9912,275.419,2973.6842,170.576,5.758191,0.087812,2.313821e+08
